In [1]:
%run utils/preparation.ipynb

In [2]:
%run utils/exploration.ipynb

In [3]:
%run utils/MLP_utils.ipynb

In [4]:
import pandas as pd
fpath = "data.csv"
df = pd.read_csv(fpath, quotechar="'") # Load csv as pandas df
df.rename( columns={
        'tumor-size': 'tumor_size',
        'inv-nodes': 'inv_nodes',
        'node-caps' : 'node_caps',
        'deg-malig' : 'deg_malig',
        'breast-quad' : 'breast_quad'
    }, inplace=True)

df = remove_missing_values(df)
df_encoded, label_encoder = encode_df(df)
X, y = get_nn_inputs(df_encoded)

In [17]:
import numpy as np
from itertools import product

max_layers = 5
max_neurons = 20
layer_space_list = []
for i in range(1, max_layers+1):
    i_layers = list(product(list(range(1, max_neurons+1)), repeat=i))
    layer_space_list.extend(i_layers) # all config: i layers
layer_space = tuple(layer_space_list)

param_grid = {
    'clf__hidden_layer_sizes' : layer_space, # (100,)
    'clf__max_iter' : (500, 800, 1000),
    'clf__activation' : ['identity', 'logistic', 'tanh', 'relu'], # 'relu'
    'clf__solver' : ['lbfgs', 'sgd', 'adam'], #'adam'
    'clf__alpha' : np.linspace(start=0.00001, stop=0.001, num=50), #0.0001
    'clf__learning_rate' : ['constant', 'invscaling', 'adaptive'], #'constant'
    'clf__learning_rate_init' : np.linspace(start=0.0001, stop=0.01, num=50), #0.001
    'clf__momentum' : np.linspace(start=0.1, stop=1, num=10) #0.9
}

In [19]:
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_grid,
    n_iter=50,
    n_jobs=8,
    refit=True,
    cv=10, # 10-fold
    verbose=1,
    random_state=None
)

random_search.fit(X, y)
print("best params:\n{}".format(random_search.best_params_))
print("best score :\n{}".format(random_search.best_score_))

Fitting 10 folds for each of 50 candidates, totalling 500 fits


/home/eolus/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/eolus/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[Parallel(n_jobs=8)]: Done 171 tasks      | elapsed:   12.5s
[Parallel(n_jobs=8)]: Done 378 tasks      | elapsed:   26.4s
/home/eolus/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/eolus/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: Converg

best params:
{'clf__solver': 'sgd', 'clf__momentum': 0.80000000000000004, 'clf__max_iter': 800, 'clf__learning_rate_init': 0.0023224489795918369, 'clf__learning_rate': 'constant', 'clf__hidden_layer_sizes': (10, 5, 1, 18), 'clf__alpha': 0.00089897959183673473, 'clf__activation': 'identity'}
best score :
0.7333333333333333
